# **This is the nnitial code to test that the program was recognizing the trained model by uploading a picture.**

In [ ]:
import tensorflow as tf
import numpy as np
import cv2  # For image or video input


In [ ]:
# Load the trained model
model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Data/keras_model.h5")

# Load the labels
with open("/content/drive/MyDrive/Colab Notebooks/Data/labels.txt", "r") as f:
    labels = [line.strip() for line in f]
print("Labels:", labels)


Labels: ['0 Like', '1 Dislike', '2 Happy', '3 High Energy', '4 Tired']


In [ ]:
def preprocess_image(image):
    # Resize the image to the expected input size of the model
    input_size = (224, 224)  # Adjust based on your model
    image_resized = cv2.resize(image, input_size)

    # Normalize the image to [0, 1]
    image_normalized = image_resized / 255.0

    # Add batch dimension (1, 224, 224, 3)
    image_batch = np.expand_dims(image_normalized, axis=0)

    return image_batch


In [ ]:
# Example: Load an image from file and preprocess it
image_path = "/content/drive/MyDrive/Colab Notebooks/Data/like.png"  # Replace with your image file path
image = cv2.imread(image_path)
input_data = preprocess_image(image)

# Get predictions
predictions = model.predict(input_data)
predicted_label_index = np.argmax(predictions)
predicted_label = labels[predicted_label_index]

# Print the result
print(f"Predicted label: {predicted_label}")
print(f"Confidence: {predictions[0][predicted_label_index]:.2f}")


1/1 [==============================] - 1s 868ms/step
Predicted label: 2 Happy
Confidence: 0.60


# **This is the code to recognize the gestures using the webcam and sending the messages through Flask server.**

In [ ]:
# Import the necessary library
from google.colab import drive

# Mount the Google Drive at /content/drive
drive.mount('/content/drive')

# Verify by listing the files in the drive
!ls /content/drive/My\ Drive/

Mounted at /content/drive
 031A68DE-B8E9-4456-B9CF-889AC71491EB.jpeg
 425F523D-FF90-4FCF-9508-408787023444.jpeg
 8E6D9982-02D3-40F3-A67D-0FEC93E095D4.jpeg
'Aliena · SlidesCarnival.gslides'
'APP MOVIL UNIVERSAL.pptx'
'besties .mov'
'Bigot- Maia Arrais.gslides'
 C0D31A87-4F1A-4517-BD7D-AAC6DDFFA934.jpeg
'CART 346 Final Project Documentation.gdoc'
'CART 346 Presenntation Info.gdoc'
'CART 346 Presentation.gslides'
 CC69D604-C589-49F4-BCC4-DD2463A1F9D0.jpeg
 certificadoPDF.pdf
'changed videos'
'Coding Exercises .gdoc'
'Colab Notebooks'
'Copy of Artificial Intelligence (AI) Startup Business Plan by Slidesgo.gslides'
'Copy of Sketchnotes Lesson by Slidesgo.gslides'
'Copy of The Fishing Formula Business Plan - Blank.gdoc'
'Critical Refelction 1- Maia Arrais.gdoc'
'Critical Reflection 2- Maia Arrais.gdoc'
'Critical Reflection 3- Maia Arrais.gdoc'
'Critical Review- Maia Arrais.gdoc'
'Descriminalización de las drogas- RCC.gdoc'
 E4960EAF-681F-4CA9-9B31-7511790D8834.jpeg
'Eduardo Pitts '
'Entrevi

In [ ]:
!pip install flask-cors pyngrok
!pip install tensorflow

In [ ]:
!pip install ngrok flask-ngrok
!pip install tensorflow==2.12.0
import tensorflow as tf
import numpy as np
import cv2
from flask import Flask, request, jsonify
from flask_cors import CORS
from flask_ngrok import run_with_ngrok  # For running Flask in Colab with ngrok
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Ngrok authentication token
NGROK_AUTH_TOKEN = "2pSo4UPWuIzDDM1ifCNOJumGz79_7WdZsxyvBcZczxPTWV7Z8"  # Replace with your token
from pyngrok import ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start Ngrok
public_url = ngrok.connect(5000)
print(f"Flask server is running at {public_url}")

# Load the pre-trained model
model_path = "/content/drive/MyDrive/Colab Notebooks/Data/keras_model.h5"
labels_path = "/content/drive/MyDrive/Colab Notebooks/Data/labels.txt"

try:
    model = tf.keras.models.load_model(model_path)
    with open(labels_path, "r") as f:
        labels = [line.strip() for line in f]
except Exception as e:
    print(f"Error loading model or labels: {e}")
    raise

# Function to preprocess images
def preprocess_image(image):
    input_size = (224, 224)  # Adjust to the model's expected input size
    image_resized = cv2.resize(image, input_size)
    image_normalized = image_resized / 255.0  # Normalize pixel values
    return np.expand_dims(image_normalized, axis=0)

# Create Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Enable ngrok for Colab
CORS(app)  # Allow cross-origin requests

@app.route("/predict", methods=["POST"])
def predict():
    try:
        # Receive the file from the request
        file = request.files.get("frame")
        if not file:
            return jsonify({"error": "No file provided"}), 400

        # Read and decode the image
        image = np.frombuffer(file.read(), np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        if image is None:
            return jsonify({"error": "Invalid image format"}), 400

        # Preprocess the image and make predictions
        input_data = preprocess_image(image)
        predictions = model.predict(input_data)
        predicted_label_index = np.argmax(predictions)
        predicted_label = labels[predicted_label_index]

        # Return prediction as JSON
        return jsonify({"prediction": predicted_label})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run the server
if __name__ == "__main__":
    app.run()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Flask server is running at NgrokTunnel: "https://cba4-34-106-161-3.ngrok-free.app" -> "http://localhost:5000"


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://cba4-34-106-161-3.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [28/Nov/2024 18:53:20] "POST /predict HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [28/Nov/2024 18:53:20] "POST /predict HTTP/1.1" 500 -


...

INFO:werkzeug:127.0.0.1 - - [28/Nov/2024 18:54:04] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 57ms/step


INFO:werkzeug:127.0.0.1 - - [28/Nov/2024 18:54:05] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 114ms/step


INFO:werkzeug:127.0.0.1 - - [28/Nov/2024 18:54:07] "POST /predict HTTP/1.1" 200 -


## **This is the code to recognize the gestures using the webcam and sending the messages through Flask server with more confidence predictability.**

In [9]:
# Install necessary packages
!pip install ngrok flask-ngrok flask-cors tensorflow==2.12.0 opencv-python-headless flask pyngrok

import tensorflow as tf
import numpy as np
import cv2
from flask import Flask, request, jsonify
from flask_cors import CORS
from flask_ngrok import run_with_ngrok
from google.colab import drive
from pyngrok import ngrok

# Mount Google Drive
drive.mount('/content/drive')

# Ngrok authentication token
NGROK_AUTH_TOKEN = "2poDpaDCLyQlCLt9YEBIpldKGv3_4NXarUYwQWsXaRnqcKNqZ"  # token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start Ngrok
public_url = ngrok.connect(5000)
print(f"Flask server is running at {public_url}")

# Load the pre-trained model and labels
model_path = "/content/drive/MyDrive/Colab Notebooks/Data/keras_model.h5"
labels_path = "/content/drive/MyDrive/Colab Notebooks/Data/labels.txt"

try:
    model = tf.keras.models.load_model(model_path)
    with open(labels_path, "r") as f:
        labels = [line.strip() for line in f]
except Exception as e:
    print(f"Error loading model or labels: {e}")
    raise

# Function to preprocess images
def preprocess_image(image):
    input_size = (224, 224)  # model's expected input size
    image_resized = cv2.resize(image, input_size)
    image_normalized = (image_resized / 127.5) - 1  # Normalize to [-1, 1]
    return np.expand_dims(image_normalized, axis=0)

# Create Flask app
app = Flask(__name__)
run_with_ngrok(app)
CORS(app)

@app.route("/predict", methods=["POST"])
def predict():
    try:
        # Receive the file from the request
        file = request.files.get("frame")
        if not file:
            return jsonify({"error": "No file provided"}), 400

        # Read and decode the image
        image = np.frombuffer(file.read(), np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        if image is None:
            return jsonify({"error": "Invalid image format"}), 400

        # Preprocess the image and make predictions
        input_data = preprocess_image(image)
        predictions = model.predict(input_data)
        predicted_label_index = np.argmax(predictions)
        predicted_label = labels[predicted_label_index]
        confidence_score = predictions[0][predicted_label_index]

        # Return prediction as JSON
        return jsonify({
            "prediction": predicted_label,
            "confidence_score": f"{confidence_score * 100:.2f}%"
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/camera_predict", methods=["GET"])
def camera_predict():
    try:
        # Open webcam
        camera = cv2.VideoCapture(0)
        if not camera.isOpened():
            return jsonify({"error": "Could not access webcam"}), 500

        ret, frame = camera.read()
        if not ret:
            return jsonify({"error": "Failed to capture image from webcam"}), 500

        # Preprocess the frame
        input_data = preprocess_image(frame)
        predictions = model.predict(input_data)
        predicted_label_index = np.argmax(predictions)
        predicted_label = labels[predicted_label_index]
        confidence_score = predictions[0][predicted_label_index]

        camera.release()

        return jsonify({
            "prediction": predicted_label,
            "confidence_score": f"{confidence_score * 100:.2f}%"
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Run the server
if __name__ == "__main__":
    app.run()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Flask server is running at NgrokTunnel: "https://34d4-34-173-117-20.ngrok-free.app" -> "http://localhost:5000"


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://34d4-34-173-117-20.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 18:38:17] "POST /predict HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 18:38:17] "POST /predict HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 18:38:17] "POST /predict HTTP/1.1" 500 -


...

INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 18:43:26] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 69ms/step


INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 18:43:27] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 40ms/step


INFO:werkzeug:127.0.0.1 - - [05/Dec/2024 18:43:28] "POST /predict HTTP/1.1" 200 -
